# 01 - the random library

1. Make sure to use Python 3, not 2.
2. For full documentation, see https://docs.python.org/3/library/random.html


In [1]:
import random

# Set the random seed for the same results each time
random.seed(1)
print("Uniformly distributed random variate between 0 (inclusive) and 1 (exclusive):", random.random())

# sample(seq,k) chooses k samples from seq without replacement
print("Random ordering of 0..5:", random.sample(range(6),6))

# Random coin flip
print("Random coin flip:", random.choice(['heads', 'tails']))

# Random sequence of 10 coin flips
# choices(seq,k) chooses k samples from seq with replacement
print("Random coin flip:", random.choices(['H', 'T'], k=10))

# Exponential distribution
print("Exponential random variates with mean 0.25:",
      random.expovariate(4),
      random.expovariate(4),
      random.expovariate(4),
      random.expovariate(4))
print("Exponential random variates with mean 4:",
      random.expovariate(1/4),
      random.expovariate(1/4),
      random.expovariate(1/4),
      random.expovariate(1/4))

Uniformly distributed random variate between 0 (inclusive) and 1 (exclusive): 0.13436424411240122
Random ordering of 0..5: [0, 2, 5, 1, 4, 3]
Random coin flip: tails
Random coin flip: ['T', 'H', 'H', 'T', 'H', 'T', 'H', 'H', 'T', 'H']
Exponential random variates with mean 0.25: 0.7263389956826974 0.5792406327290851 0.007766905615767414 0.0064438014753491315
Exponential random variates with mean 4: 3.118416425887186 11.19731880347099 1.9199200341692508 0.9764443527828108


# 02 - A Poisson process in simpy


A Poisson process is defined by three conditions:
1. The cumulative number of events at time 0 is zero.
2. The number of events in two disjoint intervals are independent.
3. The number of events in an interval of length t is a Poisson
   random variable with mean lambd*t (we would use lambda but
   that is a reserved keyword in Python).
We use the fact that inter-arrivals in a Poisson process are
independent and exponentially distributed with mean 1/lambd.


In [2]:
import simpy
import random

# Poisson process generator
def generator(env, lambd):
    nArrivals = 0
    while True:
        yield env.timeout(random.expovariate(lambd))
        nArrivals += 1
        print(nArrivals, env.now)

env = simpy.Environment()
env.process(generator(env, 1))
print('arr','time')
env.run(until=20)

arr time
1 0.5483831185108635
2 0.5778539358253091
3 0.8284864532970078
4 1.4045398910871323
5 2.089346434431785
6 2.3547250227469223
7 2.6172157992199994
8 2.864115605732936
9 3.4795676919318552
10 3.821750463089402
11 3.843474434356577
12 5.661031676965104
13 6.47398616660909
14 7.502031040245637
15 7.707710807391278
16 12.606368173050699
17 14.572099165166467
18 14.700944366364546
19 15.105452710357694
20 16.38373394440603
21 17.62572631849052


# 03 - An M/M/k queue with abandonment in simpy


An M/M/k queue has Poisson arrivals, exponentially distributed
service times, and k servers.  Requests will remain in the queue
until served or until some exponentially distributed 'patience'
time has elapsed.

Note that queueing systems with Poisson arrivals exhibit the
"PASTA" property (Poisson arrivals see time averages).  For example,
the mean queue length seen by new arrivals (excluding the new arrival)
is the mean queue length over time.
For a regular M/M/k queue, set the patience paramter to float('inf').
See Section 9 of the Zukerman textbook
(http://www.ee.cityu.edu.hk/~zukerman/classnotes.pdf)
for more on the M/M/k queue, including formulas for the mean wait time
and mean buffer length.

Parameters:
lambd - arrival rate (again we would use lambda but it is a reserved
        keyword in Python
tau   - mean service rate
patience - mean request patience


In [3]:
!pip install runstats

In [4]:
import simpy
import random
from runstats import Statistics

class SimStats:
    def __init__(self):
        self.nArrived = 0
        self.nAbandoned = 0
        self.wait = Statistics() # wait of requests that do not abandon
        self.waitAbandoned = Statistics() # wait of requests that do abandon
        self.waitAll = Statistics() # wait of all requests
        self.bufferLen = Statistics() # queue buffer length seen by new requests

# Poisson process generator
def generator(env, stats, q, lambd, tau, patience):
    while True:
        yield env.timeout(random.expovariate(lambd))
        r = request(env,
                    stats,
                    q,
                    serviceTime = random.expovariate(1/tau),
                    patience = random.expovariate(1/patience))
        env.process(r)

# Request handling
def request(env, stats, q, serviceTime, patience):
    stats.nArrived += 1
    if (stats.nArrived % 100_000 == 0):
        print(stats.nArrived,'arrivals at time',env.now)
    arrTime = env.now

    # q.users returns requests in service, q.queue returns requests in the buffer
    stats.bufferLen.push(len(q.queue))

    with q.request() as req:
        results = yield req | env.timeout(patience)
        t = env.now - arrTime
        stats.waitAll.push(t)

        if req in results: # we got service
            stats.wait.push(t)
            yield env.timeout(serviceTime)

        else: # we abandoned
            stats.waitAbandoned.push(t)
            stats.nAbandoned += 1

        # resource unit automatically released here

env = simpy.Environment()
stats = SimStats()
q = simpy.Resource(env, capacity = 10)
env.process(generator(env, stats, q, lambd = 9, tau = 1, patience = 0.1))
env.run(until=100_000)

print('Abandonment rate:', stats.nAbandoned/stats.nArrived)
print('Mean wait of served/in service requests:', stats.wait.mean())
print('Mean wait of abandoned requests:', stats.waitAbandoned.mean())
print('Mean wait of all requests:', stats.waitAll.mean())
print('Mean buffer length:', stats.bufferLen.mean())

100000 arrivals at time 11132.770495643414
200000 arrivals at time 22262.093186430448
300000 arrivals at time 33369.4299266081
400000 arrivals at time 44425.111519888414
500000 arrivals at time 55521.51578011671
600000 arrivals at time 66582.92667769705
700000 arrivals at time 77715.58765019113
800000 arrivals at time 88816.74031536843
900000 arrivals at time 99906.74645970738
Abandonment rate: 0.14262338939663222
Mean wait of served/in service requests: 0.00753982406723493
Mean wait of abandoned requests: 0.054739532260458905
Mean wait of all requests: 0.014271606428284475
Mean buffer length: 0.12873059977110193


# 04 -  M/M/k/k queue and the Erlang B formula

An M/M/k/k queue has Poisson arrivals, exponentially
distributed service times, k servers, and no buffer.
Any request that cannot receive service upon arrival
is immediately blocked and cleared from the queue.
The request blocking probability can be computed exactly
using the Erlang B formula.


In [5]:
import simpy
import random

In [6]:
class SimStats:
    def __init__(self):
        self.nArrived = 0
        self.nBlocked = 0

In [7]:
# Poisson process generator
def generator(env, stats, q, lambd, tau):
    while True:
        yield env.timeout(random.expovariate(lambd))
        r = request(env, stats, q,
                    serviceTime = random.expovariate(1/tau))
        env.process(r)

In [8]:
# Request handling
def request(env, stats, q, serviceTime):
    stats.nArrived += 1
    #if (stats.nArrived % 100_000 == 0):
        #print(stats.nArrived,'arrivals at time',env.now)
    arrTime = env.now

    if q.count < q.capacity:
        with q.request() as req:
            yield req # get the resource
            yield env.timeout(serviceTime) # hold the resource for the specified time
            # resource automatically released at end of "with" block
    else:
        stats.nBlocked += 1

In [9]:
def erlangB(load,nServers):
    inv = 1
    for m in range(1,nServers+1): # range does not include the last number, so we need to add 1
        inv = 1 + m/load * inv
    return 1/inv

In [10]:
if __name__ == "__main__":
    lambd = 9
    tau = 1

    env = simpy.Environment()
    stats = SimStats()
    q = simpy.Resource(env, capacity = 10)
    env.process(generator(env, stats, q, lambd, tau))
    env.run(until=100_000)

    print('Blocking probability:', stats.nBlocked/stats.nArrived)
    print('Exact solution:', erlangB(lambd*tau, q.capacity))

Blocking probability: 0.16737626751434
Exact solution: 0.16796322629158644


# 05 - Confidence intervals


If we assume that a particular distribution is normal, then
the Student's t-distribution can be used to compute
the confidence interval for the mean of the distribution.

A n% confidence interval means that the
true mean of a distribution falls within the
computed confidence interval with an estimated 95%
probability.

Confidence intervals are often used to describe the
uncertainty in sampled values such as survey data
or simulation results.


In [11]:
import numpy as np
import scipy.stats as st

def ci(data):
    return st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

def between(x,ci):
    return x >= ci[0] and x <= ci[1]

def test(mu,sigma):
    # mu and sigma are the true mean and sample deviation

    # empirical means
    m = np.empty(10)

    # generate distributions
    for i in range(10):
        s = np.random.normal(mu, sigma, 10)
        m[i] = np.mean(s)

    # Is the true mean within the C.I. of the empirical means (m)?
    return between(mu, ci(m))

if __name__ == "__main__":
    count = 0
    nTests = 10000

    for x in range(nTests):
        if test(0,1):
            count += 1
    print('Proportion of tests where the 95% confidence interval includes the true mean:', count/nTests)

Proportion of tests where the 95% confidence interval includes the true mean: 0.9502


# 06 - M/D/k/k queue


The M/D/k/k queue is similar to the M/M/k/k queue from Example 4,
except the service times are now constant (deterministic) rather
than random.


In [12]:
import simpy
import random

class SimStats:
    def __init__(self):
        self.nArrived = 0
        self.nBlocked = 0

# Poisson process generator
def generator(env, stats, q, lambd, tau):
    while True:
        yield env.timeout(random.expovariate(lambd))
        r = request(env, stats, q,
                    serviceTime = 1/tau)
        env.process(r)

# Request handling
def request(env, stats, q, serviceTime):
    stats.nArrived += 1
    #if (stats.nArrived % 100_000 == 0):
        #print(stats.nArrived,'arrivals at time',env.now)
    arrTime = env.now

    if q.count < q.capacity:
        with q.request() as req:
            yield req # get the resource
            yield env.timeout(serviceTime) # hold the resource for the specified time
            # resource automatically released at end of "with" block
    else:
        stats.nBlocked += 1

def erlangB(load,nServers):
    inv = 1
    for m in range(1,nServers+1): # range does not include the last number, so we need to add 1
        inv = 1 + m/load * inv
    return 1/inv

if __name__ == "__main__":
    lambd = 9
    tau = 1

    env = simpy.Environment()
    stats = SimStats()
    q = simpy.Resource(env, capacity = 10)
    env.process(generator(env, stats, q, lambd, tau))
    env.run(until=100_000)

    print('Blocking probability:', stats.nBlocked/stats.nArrived)
    print('Exact solution:', erlangB(lambd*tau, q.capacity))

Blocking probability: 0.16801675403832508
Exact solution: 0.16796322629158644


# 07 - M/M/k/k vs M/D/k/k


Because of insensitivity, the request blocking probability of the
M/G/k/k queue, of which the M/M/k/k and M/D/k/k queues are subtypes,
is insensitive to the shape of the service time distribution apart
from the mean.

Here, we use confidence intervals to demonstrate this for different
arrival rates.  We will plot the results in Example 8.


In [13]:
import simpy
import numpy as np
import ex04_mmkk as mmkk
import ex06_mdkk as mdkk
from ex05_confidence_intervals import ci

def mean_err(ci):
    # convert confidence interval from (min,max) to (mean,error_bound)
    return (np.mean(ci), np.max(ci) - np.mean(ci))

def sim_mmkk(lambd, tau = 1):
    mmkk_results = np.empty(10)
    for i in range(10):
        mmkk_env = simpy.Environment()
        mmkk_stats = mmkk.SimStats()
        mmkk_q = simpy.Resource(mmkk_env, capacity = 10)
        mmkk_env.process(mmkk.generator(mmkk_env, mmkk_stats, mmkk_q, lambd, tau))
        mmkk_env.run(until=100_000.0/lambd)
        mmkk_results[i] = mmkk_stats.nBlocked/mmkk_stats.nArrived
    return mean_err(ci(mmkk_results))

def sim_mdkk(lambd, tau = 1):
    mdkk_results = np.empty(10)
    for i in range(10):
        mdkk_env = simpy.Environment()
        mdkk_stats = mdkk.SimStats()
        mdkk_q = simpy.Resource(mdkk_env, capacity = 10)
        mdkk_env.process(mdkk.generator(mdkk_env, mdkk_stats, mdkk_q, lambd, tau))
        mdkk_env.run(until=100_000.0/lambd)
        mdkk_results[i] = mdkk_stats.nBlocked/mdkk_stats.nArrived
    return mean_err(ci(mdkk_results))

# lambd = 3 to 15.1 by 1 (we make the endpoint a bit bigger than 15 to ensure 15 is included in the range)
for lambd in np.arange(3,15.1,1):
    print(lambd, *sim_mmkk(lambd), *sim_mdkk(lambd))

ModuleNotFoundError: No module named 'ex04_mmkk'

# Example 08: plotting using Matplotlib


Here we plot the results of Example 7, which shows the simulation results
for an M/M/10/10 queue and an M/D/10/10 queue with respect to the offered load.
For convienience, the output of Example 7 is loaded from a text file rather
than running the simulations again.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# each column of the file is saved to a separate variable
# when unpack = True
x, y1, yerr1, y2, yerr2 = np.loadtxt('ex07_mmkk_mdkk.txt',unpack=True)

plt.errorbar(x,y1,yerr1,fmt='b',capsize=8, label='M/M/10/10')
plt.errorbar(x,y2,yerr2,fmt='r--',capsize=5, label='M/D/10/10')
plt.xlabel('λ, Arrival rate', fontsize = 14) # check Unicode support
plt.ylabel('Request blocking probability', fontsize = 14)
plt.legend(loc='upper left')
plt.show()